# Part 2. Word Vectors

[https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-2-word-vectors](https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-2-word-vectors)

<br>

# Code

The tutorial code for Part 2 lives [here](https://github.com/wendykan/DeepLearningMovies/blob/master/Word2Vec_AverageVectors.py).

<br>

# Introducing Distributed Word Vectors

This part of the tutorial will focus on using distributed word vectors created by the Word2Vec algorithm. (For an overview of deep learning, as well as pointers to some additional tutorials, see the "What is Deep Learning?" page).

Parts 2 and 3 assume more familiarity with Python than Part 1. We developed the following code on a dual-core Macbook Pro, however, we have not yet run the code successfully on Windows. If you are a Windows user and you get it working, please leave a note on how you did it in the forum! For more detail, see the "Setting Up Your System" page.

Word2vec, published by Google in 2013, is a neural network implementation(실행) that learns distributed representations for words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

Word2Vec does not need labels in order to create meaningful representations. This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing(아주 흥미로운) characteristics. Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies(유사점), can be reproduced using vector math. The famous example is that, with highly trained word vectors, "king - man + woman = queen."

Check out Google's code, writeup, and the accompanying papers. [This presentation](https://docs.google.com/file/d/0B7XkCwpI5KDYRWRnd1RzWXQ2TWc/edit) is also helpful. The original code is in C, but it has since been ported to other languages, including Python. We encourage you to play with the original C tool, but be warned that it is not user-friendly if you are a beginning programmer (we had to manually edit the header files to compile it).

Recent work out of Stanford has also [applied deep learning to sentiment analysis](http://nlp.stanford.edu/sentiment/); their code is available in Java. However, their approach, which relies on sentence parsing, cannot be applied in a straightforward(복잡하지 않은) way to paragraphs(단락) of arbitrary(임의적인) length.

Distributed word vectors are powerful and can be used for many applications, particularly word prediction and translation. Here, we will try to apply them to sentiment analysis.

<br>

# Using word2vec in Python

In Python, we will use the excellent implementation of word2vec from the `gensim` package. If you don't already have gensim installed, you'll need to install it. There is an excellent tutorial that accompanies(동반하다) the Python Word2Vec implementation, [here](http://radimrehurek.com/2014/02/word2vec-tutorial/).

Although Word2Vec does not require graphics processing units (GPUs) like many deep learning algorithms, it is compute intensive(짧은 시간에 많은 일을 하는). Both Google's version and the Python version rely on multi-threading (running multiple processes in parallel on your computer to save time). ln order to train your model in a reasonable amount of time, you will need to install `cython` ([instructions here](http://docs.cython.org/src/quickstart/install.html)). Word2Vec will run without cython installed, but it will take days to run instead of minutes.

<br>

# Preparing to Train a Model

Now down to the nitty-gritty! First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use **unlabeledTrain.tsv**, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [1]:
import pandas as pd

# Read data from files
train = pd.read_csv("../input/labeledTrainData.tsv",
                    header=0,
                    delimiter="\t",
                    quoting=3)
test = pd.read_csv("../input/testData.tsv",
                   header=0,
                   delimiter="\t",
                   quoting=3)
unlabeled_train = pd.read_csv("../input/unlabeledTrainData.tsv",
                              header=0,
                              delimiter="\t",
                              quoting=3)

In [3]:
# Verify(확인하다) the number of reviews were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews,\
 and %d unlabeled reviews.\n" % (train["review"].size,
                                      test["review"].size,
                                      unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews.



The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader(넓은) context(맥락) of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [5]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's **punkt** tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [6]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [7]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
from tqdm import tqdm

In [10]:
# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False) :
    
    # Function to split a review into parsed sentences.
    # Returns a list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    # 
    # 2. Loop over each sentence
    sentences = []
    
    for raw_sentence in raw_sentences :
        
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0 :
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
            
    # Return the list of sentences (each sentence is a list of words)
    # so this returns a list of lists
    return sentences

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

In [11]:
sentences = [] # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in tqdm(train["review"]) :
    sentences += review_to_sentences(review, tokenizer)
    
print("Parsing sentences from unlabeled set")
for review in tqdm(unlabeled_train["review"]) :
    sentences += review_to_sentences(review, tokenizer)

  0%|                                      | 56/25000 [00:00<00:44, 559.97it/s]

Parsing sentences from training set


  1%|▎                                    | 215/25000 [00:00<00:46, 528.92it/s]c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:302: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  6%|██▏                                 | 1523/25000 [00:02<00:45, 516.30it/s]c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:302: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  6%|██▎                                 | 1575/25000 [00:02<00:45, 511.29it/s]c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:375: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL,

Parsing sentences from unlabeled set


  1%|▏                                    | 259/50000 [00:00<01:26, 573.29it/s]c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:375: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
  5%|█▊                                  | 2531/50000 [00:04<01:19, 594.87it/s]c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:375: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
 14%|█████▏                              | 7182/50000 [00:12<01

You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

We can take a look at the output to see how this differs from Part 1:

In [12]:
# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))

795538


In [13]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [14]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


A minor detail to note is the difference between the "+=" and "append" when it comes to Python lists. In many applications the two are interchangeable(교환할 수 있는), but here they are not. If you are appending a list of lists to another list of lists, "append" will only append the first list; you need to use "+=" in order to join all of the lists at once.

<br>

# Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec [API documentation](https://radimrehurek.com/gensim/models/word2vec.html) as well as the [Google documentation](https://code.google.com/archive/p/word2vec/).

## Architecture

Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

## Training algorithm

Hierarchical softmax (default) or negative sampling. For us, the default worked well.

## Downsampling of frequent words

The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

## Word vector dimensionality

More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

## Context / window size

How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

## Worker threads

Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

## Minimum word count

This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

<br>

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [15]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [16]:
# Set values for various parameters
num_features = 300 # Word Vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsampling setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec

print("Training model...")
model = word2vec.Word2Vec(sentences,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)

# If you don't plan to train the model any further, 
# calling init_sims will make the model much more memory-efficientt.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-11-19 12:39:19,156 : INFO : 'pattern' package not found; tag filters are not available for English
2019-11-19 12:39:19,187 : INFO : collecting all words and their counts
2019-11-19 12:39:19,188 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-19 12:39:19,240 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-11-19 12:39:19,292 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-11-19 12:39:19,345 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types


Training model...


2019-11-19 12:39:19,403 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2019-11-19 12:39:19,458 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2019-11-19 12:39:19,511 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2019-11-19 12:39:19,557 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2019-11-19 12:39:19,611 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2019-11-19 12:39:19,657 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2019-11-19 12:39:19,729 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-11-19 12:39:19,774 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-11-19 12:39:19,838 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

On a dual-core Macbook Pro, this took less than 15 minutes to run using 4 worker threads. However, it will vary depending on your computer. Fortunately, the logging functionality prints informative messages.

If you are on a Mac or Linux system, you can use the "top" command from within Terminal (not from within Python) to see if your system is successfully parallelizing while the model is training. Type

```
top -o cpu
```

into a terminal window while the model is training. With 4 workers, the first process in the list should be Python, and it should show 300-400% CPU usage.

If your CPU usage is lower, it may be that cython is not working correctly on your machine.

<br>

# Exploring the Model Results

Congratulations on making it successfully through everything so far! Let's take a look at the model we created out of our 75,000 training reviews.

The "doesnt_match" function will try to deduce(추론하다) which word in a set is most dissimilar from the others:

In [17]:
model.doesnt_match("man woman child kitchen".split())

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

Our model is capable(유능한) of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle(미묘한) differences in meaning, such as differences between countries and cities:

In [18]:
model.doesnt_match("france england germany beerlin".split())

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2019-11-19 12:41:00,137 : WARNING : vectors for words {'beerlin'} are not present in the model, ignoring these words


'germany'

... although with the relatively small training set we used, it's certainly not perfect:

In [19]:
model.doesnt_match("paris berlin london austria".split())

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'austria'

We can also use the "most_similar" function to get insight into the model's word clusters:

In [20]:
model.most_similar("man")

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6273036003112793),
 ('lady', 0.5846002101898193),
 ('lad', 0.5529636144638062),
 ('sailor', 0.5280630588531494),
 ('farmer', 0.5279965400695801),
 ('monk', 0.5258042216300964),
 ('guy', 0.5247218012809753),
 ('men', 0.5181735754013062),
 ('person', 0.5127182006835938),
 ('millionaire', 0.5126304626464844)]

In [21]:
model.most_similar("queen")

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6583582758903503),
 ('bride', 0.5927019715309143),
 ('victoria', 0.5862059593200684),
 ('latifah', 0.5788651704788208),
 ('goddess', 0.5758845806121826),
 ('maid', 0.571242094039917),
 ('showgirl', 0.568920373916626),
 ('stepmother', 0.5686436295509338),
 ('starlet', 0.5639791488647461),
 ('mistress', 0.56321120262146)]

Given our particular training set, it's not surprising that "Latifah" is a top hit for similarity with "Queen".

Or, more relevant for sentiment analysis:

In [22]:
model.most_similar("awful") # awful : 끔직한

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.770870566368103),
 ('horrible', 0.7331541776657104),
 ('atrocious', 0.7274578809738159),
 ('dreadful', 0.7118092775344849),
 ('abysmal', 0.6986379027366638),
 ('appalling', 0.6884308457374573),
 ('horrendous', 0.6853016018867493),
 ('horrid', 0.6731444597244263),
 ('lousy', 0.6525260210037231),
 ('laughable', 0.6127434968948364)]

So it seems we have a reasonably good model for semantic meaning - at least as good as Bag of Words. But how can we use these fancy distributed word vectors for supervised learning? The next section takes a stab at that.